<a href="https://colab.research.google.com/github/cianhwang/bigpic_sr/blob/main/demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Physical Super Resolution (SR) Against Diffraction Blur

#### Qian Huang
#### 1/12/2023

**Motivation**: Fisher information suggests we can go beyond diffraction limit given prior knowledge, which can be effectively learned by neural algorithm on large datasets.

**Goal**: develop an neural algorithm to restore images impacted by diffraction blur.

**Method**:
* Develop a SR neural algorithm that can use SRCNN or EDSR, two classic SR networks, as its backbone for physical SR.
* Simulate the camera and develop an associate forward model to generate degraded images from high quality image datasets like DIV2K
* Use generated samples to train the network.
* Save the network weights that perform best on validation set and use them for future inference.


## Pre-requisite configure

In [1]:
! wget https://github.com/cianhwang/bigpic_sr/archive/refs/heads/main.zip
! unzip -q main.zip
! mv bigpic_sr-main/* ./

--2023-01-13 07:48:47--  https://github.com/cianhwang/bigpic_sr/archive/refs/heads/main.zip
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/cianhwang/bigpic_sr/zip/refs/heads/main [following]
--2023-01-13 07:48:48--  https://codeload.github.com/cianhwang/bigpic_sr/zip/refs/heads/main
Resolving codeload.github.com (codeload.github.com)... 20.205.243.165
Connecting to codeload.github.com (codeload.github.com)|20.205.243.165|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘main.zip’

main.zip                [       <=>          ]   4.33M  3.03MB/s    in 1.4s    

2023-01-13 07:48:49 (3.03 MB/s) - ‘main.zip’ saved [4536978]



In [2]:
! wget http://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_train_HR.zip
! wget http://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_valid_HR.zip
! unzip -q DIV2K_train_HR.zip
! unzip -q DIV2K_valid_HR.zip

--2023-01-13 07:48:50--  http://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_train_HR.zip
Resolving data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)... 129.132.52.178, 2001:67c:10ec:36c2::178
Connecting to data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)|129.132.52.178|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_train_HR.zip [following]
--2023-01-13 07:48:50--  https://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_train_HR.zip
Connecting to data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)|129.132.52.178|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3530603713 (3.3G) [application/zip]
Saving to: ‘DIV2K_train_HR.zip’

DIV2K_train_HR.zip  100%[===================>]   3.29G  16.1MB/s    in 3m 53s  

2023-01-13 07:52:44 (14.5 MB/s) - ‘DIV2K_train_HR.zip’ saved [3530603713/3530603713]

--2023-01-13 07:52:44--  http://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_valid_HR.zip
Resolving data.vision.ee.ethz.ch

In [3]:
! pip install tensorboardX

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.4/125.4 KB 12.5 MB/s eta 0:00:00


## Train & Validate model

Train on DIV2K_train RandomCrop(256) and validate on DIV2K_val_CenterCrop(512)

command arguments: 

- lam: working wavelength in meters
- n_photon: the number of photons that brightest pixel received
- f-num: F-number of the lens
- p: pixel pitch on the sensor in meters

In [4]:
! python train.py --train-file DIV2K_train_HR --eval-file DIV2K_valid_HR --outputs-dir outputs_temp --num-epochs 3 --n_photon 1000 --f_num 9 --kernel jinc --lam 0.55e-6 --p 3e-6

[*] Saving outputs to outputs_temp/1000x9xjinc
[*] Saving tensorboard logs to runs/1000x9xjinc
/content/kernels.py:6: RuntimeWarning: invalid value encountered in true_divide
  f = j1(rho)/rho
/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
epoch 1/3: 100% 800/800 [02:01<00:00,  6.57it/s, loss=0.015289]
validate: 100% 100/100 [00:31<00:00,  3.22it/s, psnr=27.70, ssim=0.8266]
epoch 2/3: 100% 800/800 [01:53<00:00,  7.07it/s, loss=0.001309]
validate: 100% 100/100 [00:32<00:00,  3.09it/s, psnr=29.70, ssim=0.8484]
epoch 3/3: 100% 800/800 [01:51<00:00,  7.16it/s, loss=0.001

## Evaluate pretrained model

on DIV2K_val_CenterCrop(1024)

In [5]:
! python eval.py --eval-file DIV2K_valid_HR --model_path outputs/1000x9xjinc --n_photon 1000 --f_num 9 --kernel jinc --lam 0.55e-6 --p 3e-6 --is_pred

/content/kernels.py:6: RuntimeWarning: invalid value encountered in true_divide
  f = j1(rho)/rho
100% 100/100 [02:06<00:00,  1.26s/it, psnr=33.09, ssim=0.9146]
1000x9xjinc psnr: 33.09 | ssim: 0.9146


## Test pretrained model

In [6]:
! python test.py --weights-file outputs/1000x9xjinc --image-file DIV2K_valid_HR --output-path test/ --n_photon 1000 --f_num 9 --kernel jinc --lam 0.55e-6 --p 3e-6

/content/kernels.py:6: RuntimeWarning: invalid value encountered in true_divide
  f = j1(rho)/rho
  3% 3/100 [00:06<03:02,  1.88s/it]small image: 0804 skipped
  5% 5/100 [00:09<02:32,  1.60s/it]small image: 0807 skipped
 27% 27/100 [00:40<01:42,  1.41s/it]small image: 0830 skipped
 48% 48/100 [01:10<01:14,  1.43s/it]small image: 0852 skipped
 49% 49/100 [01:11<01:13,  1.44s/it]small image: 0854 skipped
 71% 71/100 [01:42<00:41,  1.42s/it]small image: 0877 skipped
 74% 74/100 [01:47<00:37,  1.43s/it]small image: 0881 skipped
 92% 92/100 [02:12<00:11,  1.39s/it]small image: 0900 skipped
 92% 92/100 [02:12<00:11,  1.44s/it]
